In [1]:
import json
import os
from tqdm import tqdm
from PACKAGE import multi_rouge
from rouge_score.scoring import AggregateScore, Score
from PACKAGE import metric_realization
import openai
from langchain.chat_models import ChatOpenAI
import numpy as np

# Load api_key

In [4]:
with open('../../keys.json', 'r') as f:
    api_keys = json.load(f)
api_key = api_keys[4]

# Calculate tokens

In [15]:
def calculate_tokens(articles: list):
    llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k-0613", openai_api_key=api_key, temperature=0.7, max_tokens=600)
    return [llm.get_num_tokens(str(article)) for article in articles]


path = 'QMSum/oracle'
model_name = 'bart'

# Load pred
pred_file = model_name + '_summary.json'
file_path = os.path.join(path, 'summary/' + pred_file)
with open(file_path, 'r') as f:
    predictions = json.load(f)

# Load ref
ref_file = 'test.json'
file_path = os.path.join(path, ref_file)
if os.path.exists(file_path):
    with open(file_path, 'r') as f:
        references = json.load(f)
    references = [
        data_item['Summary']
        for data_item in references]

with open('SQuALITY/LLM-embedding/max/test.json','r') as f:
    data=json.load(f)
    data=[item['Query']+item['Article']for item in data]
print(np.mean(calculate_tokens(data)))
# print(calculate_tokens(predictions))
# print(calculate_tokens(references))

8822.588461538462


In [ ]:
# Get prompt
metric_list = ['coh', 'con', 'flu', 'rel']
metric_type = metric_list[1]
prompt = open('GPTeval/prompts/' + metric_type + '_detailed.txt').read()
# Get messages
messages = []
for index, prediction in enumerate(predictions):
    reference = references[index]
    cur_prompt = prompt.replace('{{Document}}', reference).replace('{{Summary}}', prediction)
    messages.append([{"role": "system", "content": cur_prompt}])
print(len(predictions))
print(np.mean(calculate_tokens(messages)))

# Print rouge squality score

In [7]:
# Print rouge score
for root, dirs, files in os.walk('QMSum'):
    for file in files:
        if file == 'gpt3_truncate_squality_rouge.json':
            with open(os.path.join(root, file), 'r') as f:
                rouge = f.read()
                obj_rouge = eval(rouge)
                parts = root.split("\\")
                print(parts[1]+'\\'+parts[2])
                print(f"rouge1:{obj_rouge['rouge1'].mid.fmeasure * 100:.2f}")
                print(f"rouge2:{obj_rouge['rouge2'].mid.fmeasure * 100:.2f}")
                print(f"rougeL:{obj_rouge['rougeL'].mid.fmeasure * 100:.2f}")
                print()


dense\MIN
rouge1:31.19
rouge2:7.54
rougeL:16.77

LLM-embedding\MIN
rouge1:33.32
rouge2:8.45
rougeL:18.06

oracle\evaluation
rouge1:36.74
rouge2:10.09
rougeL:19.20

sparse\MIN
rouge1:29.00
rouge2:5.74
rougeL:15.65



# Print rouge score

In [ ]:
# Print rouge score
for root, dirs, files in os.walk('QMSum'):
    for file in files:
        if file == 'pri_rouge.json':
            with open(os.path.join(root, file), 'r') as f:
                rouge = json.load(f)
            for key, value in rouge.items():
                parts = root.split("\\")
                print(parts[1]+'\\'+parts[2])
                print(key)
                print(value)

# Print bert score

In [ ]:
# Print bert score
for root, dirs, files in os.walk("QMSum"):
    for file in files:
        if file == "bart_bert_score.json":
            with open(os.path.join(root, file), "r") as f:
                bert = json.load(f)
            average_f1 = bert['average_f1']
            parts = root.split("\\")
            print(parts[1]+'\\'+parts[2])
            print(f"{average_f1 * 100:.2f}")

# Print GPT eval

In [6]:
import os
import json
metric_list = ['coh', 'con', 'flu', 'rel']
metric_index = 3
model_list = ['bart_', 'gpt3_','llama_','pri_']
model_index = 1
suffix = model_list[model_index] + metric_list[metric_index] + '_truncate_gpteval.json'
for root, dirs, files in os.walk("QMSum"):
    for file in files:
        if file == suffix:
            with open(os.path.join(root, file), "r") as f:
                gpteval = json.load(f)
            value_li = []
            for key, value in gpteval.items():
                value_li.extend(value)
            # average = sum(value_li) / len(value_li)
            average = gpteval['average'][0]
            # gpteval['average'] = [average]
            print(os.path.join(root, file))
            print(f'{average * 10:.2f}')
            # with open(os.path.join(root, file), "w") as f:
            #     temp = json.dumps(gpteval)
            #     f.write(temp)

QMSum\dense\MIN\evaluation\gpt3_rel_truncate_gpteval.json
28.23
QMSum\LLM-embedding\MIN\evaluation\gpt3_rel_truncate_gpteval.json
30.85
QMSum\oracle\evaluation\gpt3_rel_truncate_gpteval.json
32.92
QMSum\sparse\MIN\evaluation\gpt3_rel_truncate_gpteval.json
26.92


# Rename

In [ ]:
# rename
for root, dirs, files in os.walk("../ODMDS_share_evaluate/SQuALITY"):
    if root.endswith('summary'):
        prefix = 'newSummary_'  # 要替换的前缀
        new_prefix = 'gpt3_summary_'  # 新的前缀
        for filename in files:
            if filename.startswith('newSummary_'):
                new_filename = new_prefix + filename[len(prefix):]
                os.rename(os.path.join(root, filename), os.path.join(root, new_filename))
    if root.endswith('evaluation'):
        new_prefix = 'gpt3_'  # 新的前缀
        for filename in files:
            if filename.startswith('evaluate'):
                new_filename = new_prefix + filename
                os.rename(os.path.join(root, filename), os.path.join(root, new_filename))
    for filename in files:
        if filename == 'all_results.json':
            os.remove(os.path.join(root, filename))

# Delete file

In [ ]:
# rename
for root, dirs, files in os.walk("../ODMDS_share_evaluate"):
    for filename in files:
        if 'evaluate_' in filename:
            file_path = os.path.join(root, filename)
            os.remove(file_path)

# Create file

In [ ]:
for root, dirs, files in os.walk("QMSum"):
    if not files and not dirs:
        with open(os.path.join(root, 'empty.txt'), 'w') as f:
            f.write('1')

# Test random index of primera

In [ ]:
with open('QMSum/randomIndex/index.json', 'r') as f:
    random_index_list = json.load(f)
for root,dirs,files in os.walk('QMSum'):
    for filename in files:
        filepath=os.path.join(root,filename)
        if 'pri_rel' in filename:
            with open(filepath,'r') as f:
                rel=json.load(f)
            rel=[rel['Summary'][index] for index in random_index_list]
            print(np.mean(rel))